## Partie Analyse des données

#### Présentation du programme
Le projet à réaliser consiste à réaliser le positionnement de sous-séquences, appellées "reads", sur un génome. Afin de gagner en précision et de pallier aux mutations engendrées par le séquençage des reads, le paramètre k permet de choisir la taille de découpage de la séquence. Pour gagner en efficacité, le programme se base sur l'algorithme DC3 et sur la Burrows Wheeler Transform. Enfin les reads sont reconstruits selon 2 méthodes, la premières identifiera les positions susceptibles de présenter le read en se basant sur le premier et dernier morceau, ensuite ceux-ci seront analysés afin de trouver les positions exactes des sous-séquences. Si aucune position exacte n'est trouvé, il y a du alors y avoir mutation et le fichier sortira la position avec un ~ devant le nombre.

#### Fichiers de sortie et temps estimé
Les résultats de l'algorithme se retrouvent après lancement dans le fichier result.txt. Nous avons regroupé les résultats obtenus sur les reads du premier chromosome dans le fichier result_all.txt. Ce fichier ne regroupe pas tous les résultats des algorithmes, car nous avons lancé sur plusieurs machines afin de pallier au temps estimé d'exécution du programme, (qui est de 75 jours pour un k = 10 !) mais n'avons obtenu qu'un seul chromosome complet.(Au vu des résultats des morceaux obtenus, le résultat prédit par tqdm est assez robuste même si le temps peut varier très fortement entre les différents reads, on se retrouve en moyennes sur le temps prédit après stabilistaion sur quelques reads courts). En augmentant k à 50, on divise le temps de calcul par 5, certes, mais on perd en précision comme le montrera l'analyse des données.

L'algorithme est cependant assez gourmand en mémoire, afin de lancer l'algorithme avec le calcul de DC3 sans passer par l'import de données depuis le fichier texte il est conseillé de pouvoir allouer plus de 10G de RAM au notebook. Sinon la version import permet d'outre-passer cette barrière (un PC "classique" à 8G pourra le faire tourner sans problème).

#### Disclaimer à propos des données :
Ce processus a necessité de recoder notre algorithme pour le faire fonctionner en "mode partition", ce qui a induit certaines erreurs de classification dans la sortie des premiers reads lancés, ainsi pour résumer : Les 10000 premiers (de NC_004325.2-100000 à NC_004325.2-90000) ont été recalculé et sont donc correct, cependant la tranche 10000-40000 et 60000-70000 contient des erreurs des classifications (certaines positions n'ont pas la ~ correspondant à une mutation à tort !) et certaines positions vers 200000 également. Cependant en laçant le programme actuellement, ce soucis ne se produira pas. Ces séquences seront écartées pour le choix de l'échantillon car elles ne permettent pas une distinction assez précise pour la quailté mais reste un bon indicateur de précision du mapping et seront comptés ainsi.

#### Etude des données
Comme il y a beaucoup de données, les vérifications de la validité de l'algorithme se feront sur l'échantillon 0-10000 et on tirera aléatoirement 200 résultats à analyser pour généraliser à l'ensemble du jeu de données. Nous avons pris le choix d'étendre l'échantillon à 250 afin de voir le gain en précision bien que la p-value soit déja assez significatif pour l'hypothèse de représentaivité des données. (Voir fichier Stat_R.jpeg) 

L'étude à ensuite été élargie sur tous les autres intervalles obtenus du génome et, en sélectionnant 10 reads par chromosome, a constitué un nouvel échantillon pour tester l'efficacité globale de l'algorithme.

Les annotations réalisées et les divers comptages et pourcentages ont été recensés dans les fichiers présents dans le dossier "Interpretation manuelle" du dossier "result_save".

#### Interprétations des résultats
Tout d'abord, le choix de k est un paramètre très important nous l'avons fixé à 10 (choisi avant la comparaison mais on connaissait déja l'influence qu'aurait ce paramètre) car il permet d'obtenir des résultats plus significatif en diminuant sa valeur (47% de positions mappés pour k = 50 contre 69% pour k = 25 et 87% pour k = 10). Ainsi bien qu'encore très long à calculer cela nous aura permis d'analyser des résultats de plutôt bonne qualité.

Les résultats obtenus sur le premier chromosome nous montrent qu'en comparaison avec le fichier BAM distribué, on retrouve près de 80% de positions prédites identiques. Dans les 20% restant se trouvent les positions qui n'ont pas été mappées (~13%), quelques positions qui prédisent la bonne position, mais avec d'autres positions qui ont le même degré de certitude donc que notre algorithme ne saura pas trier/mettre en avant, et une position qui diffèrent du fichier BAM. Cette dernière s'explique à la façon dont à été codé notre algorithme : on regarde la première et la dernière position, or dans le cas du fichier BAM, les extrémités de la séquence prédite n'étaient pas totalement couverte (moins de 10 nucléotides en amont et aval) mais contenait une excellente couverture du read sur le milieu (60 qui s'enchainaient).

Concernant l'extrapolation à l'ensemble des données, on observe que globalement on observe que le pourcentage de correctement mappées diminuent légérement au profit des mal mappées et des incertaines. Mais impossible de tirer de vraies conclusions sachant que le premier échantillon étant plus gros n'avait pas réussi à être représentatif.


## Partie Code :

### Définition des paramètres :

In [ ]:
k = 10 # On défini k, la longueur de chaque kmers
by_DC3 = False # True : on calcule la DC3 depuis les données, False : on la récupère depuis le fichier de sauvegarde
start = 0 # On commence par le premier read sinon modifier ligne 12

### Import des librairies

In [1]:
# Librairies nécessaire à l'exécution du programme
from Bio import SeqIO # Permet l'import de la fonction parse du module pour lire les fichiers fasta/fastq
from Bio.Seq import Seq # Permet la transformation en complémentaire inverse de séquences
from Bio.SeqIO.QualityIO import FastqGeneralIterator # Permet un parse plus rapide lorsque beaucoup de séquences (fastaq uniquement sinon voir site biopython pour fasta)
import functools as ft # Permet de regrouper des listes pour radix sort
from collections import Counter # Permet de compter les occurences d'un charactère dans un texte de manière efficace
import numpy as np # DC3 utilise un tableau numpy pour stocker ses données

# Librairies optionelles à l'éxécution du programme
from tqdm import tqdm # Permet d'estimer le temps d'éxécution sur un boucle
from datetime import datetime # Permet de comparer la vitesse de 2 programmes 
#%load_ext snakeviz # Permet d'optimiser le temps d'execution

### Fonctions d'import des données :

Complexité des algorithms : 

Tous : linéaire en temps avec le nombre d'éléments à importer (ou nombre d'élément fois taille du read/k pour import_read)

In [2]:
def import_DC3(filename) :
    """
    Fonction qui importe depuis un fichier texte la table des suffixes du génome par DC3 et la stocke dans une liste
    """
    filename = "DC3_save/" + filename
    with open(filename, 'r') as f: # On ouvre le fichier et on fait la procédure inverse au stockage
        text = f.readline()
        data = text.split(" ")[:-1]
        for i in range(len(data)) :
            data[i] = int(data[i])
        return(data)
    # La fonction d'export de DC3 peut nottament être retrouve dans Association_codes.ipynb
    
def genome_import() :
    """
    Fonction qui importe depuis un fichier fasta la séquence du génome dans une liste contenant ["seq", "n°k"]
    """
    list_genom=[]
    for record in SeqIO.parse("GCF_000002765.5_GCA_000002765_genomic.fna","fasta"):
        list_genom.append([str(record.seq).upper(),record.description[-14:]]) # attention le marquage est à changer pour la dernière séquence
    
    return list_genom
    
def reads_import_cuts(k) :
    """
    Fonction qui importe depuis un fichier fasta les séquences des reads et les coupes en les rangeant
    dans une liste contenant ["seq", "nom", "n°kmer"]
    
    Entrée : k, int : longueur du kmer
    
    Sortie : list_reads : liste de tous les read découpés en kmers
    """
    list_reads=[]
    with open("single_Pfal_dat.fq") as in_handle:
        for title, seq, qual in tqdm(FastqGeneralIterator(in_handle), desc = "Initialisation - Import sequence"):
            i = 1    # Incrémenteur du nombre de k-mer
            while len(seq) >= 1 : # On parcoure toute la séquence
                if len(seq) > k : # Cas où le k-mer est entier
                    list_reads.append([str(seq[0:k]).upper(),title, i])
                    i += 1
                    seq = seq[k:]
                else : # Cas où le dernier k-mer n'est pas entier
                    list_reads.append([str(seq).upper(),title, i])
                    seq = ""
                    i += 1 # Pour avoir le bon nombre de kmer s'il en reste un dernier
            
    return list_reads, i-1 # i-1 car on incrémente après avoir annoté

### Fonction pour la table des suffixes (DC3)

Complexité des algorithms : linéaire avec la longueur de la séquence pour DC3 et avec la taille de la liste pour les autres

In [3]:
def flatten(arr):
    """
    Permet de "compresser" une liste 2D en liste 1D (toutes les sous listes à la suite)
    """
    return ft.reduce(lambda x, y: x + y, arr)

def counting_sort(array,digit,p): 
    """
    Le counting sort est un tri stable utilisé par le tri radix
    Ici la fonction est adaptée au format de données (triplets de nombres à plusieurs digits)
    
    Entrée :
        array (list) : array à trier
        digit (int) : digit à regarder (unités, dizaines...)
        p (int) : numero de l'élement des triplets a regarder
    
    Sortie :
        array rangé selon le digit et la position p donnée
    """
    ##The counting sort is a stable sort used in the radic sort
    ##here the counting sort needs to be adapted to look at only one digit of each number (for radix)
    ##we also added the parameter p to be able to read the triplets (more info below)
    ##1) since we only sort single digits, the max will always be smaller than 10
    ##2) create count index (that will have the cumulative index in the end)
    count_list = [[]for i in range(10)]
    for tpl in array :
        n_uplet = tpl[0]##we take the first element of the tuple, the n-uplet
        num = n_uplet[p] // digit ##we cut off the digits to the right by getting the quotient of the euclidian division
        ##p is the index of the number studied in the triplet
        count_list[num%10].append(tpl)##we cut off the digits to the left by getting the remainder of the euclidian division
    arr_ord = flatten(count_list)

    for i in range(len(array)):##we change the base array to allow the radix sort to loop easily
        array[i] = arr_ord[i]
        
def radix_sort(array,p):
    """
    Prend en entrée un tableau de valeur de valeur et le trie en foncion de la première composante
    selon l'algorithm du radix sort
    
    Entrée : array à trier
             p : nombre de cases à trier p-uplets
    
    Sortie : array trié
    """
    ##Here the radix sort is modified to work with the triplet list sent by the DC3
    ##The code is not flexible enough to compute all characters in the ascii table, but it's enough for the use needed
    ##1) we search for the max in the nuplets
    mx = (max(array)[0])[0]+1
    if p != 3 : ##we take max = 100 because A T C G are all below 100 in ascii code
        for tpl in array:
            n_uplet = tpl[0]
            if n_uplet[-1] > mx :
                mx = n_uplet[-1]+1
    '''##2) to know how many loops we have to do, we will use a variable to represent,
    the digit we are currently in'''
    for i in reversed(range(0,p)):
        digit = 1 ##starts at one for units
        while mx - digit > 0 :##when all the digits are checked, digit will be greater than the maximum
            counting_sort(array,digit,i)
            digit *= 10 ##digit will go to the tens, the hundreds, the thousands...
            
def DC3(S, P_12_base = []) : # On initialise P_12_base comme étant vide car elle n'est pas utilisé pour la première récursion
    """
    Create the suffix array from DC3 algorithm, could be recursive if needed
    
    Args:
        S (str): string
        P_12_base : store P1+2 from recursion to map correctly recursivity
    
    Return:
        index_012 : suffix array of S
        order_12 : order of the next recursion to map correctly recursivity
    """
    DC3_table = np.zeros((3,len(S) + 3), dtype=int) # Les caractères sentinelles sont déja là !
    """
    Table de DC3 qui contient en chaque ligne : 
    Ligne 0 : indice du caractère, optionelle en soit mais permet un debugging plus efficace
    Ligne 1 : conversion du caractère en nombre
    Ligne 2 : Ordre de l'indice du caractère
    
    """
    for i in range(len(S) + 3) :
            DC3_table[0][i] =  i  # On initialise les indices
    
    # String conversion : !!!!! à n'executer que lors de la première récursion !!!!!
    if type(S) == str :
        S_l = [*S] # On sépare caractère par caractère : "ATGC" devient ["A","T","G","C"]
        for i in range(len(S_l)) :
            DC3_table[1][i] =  ord(S_l[i]) # On rempli le caractère par son code Ascii dans la table
    else :
         for i in range(len(S)) :
            DC3_table[1][i] =  S[i] # Cas où l'on rentre dans la boucle une deuxième fois ou plus, pas de conversion
            
    # On crée P0, P1, P2 et P1+P2 :    
    P0 = [*range(0,len(S)+1,3)] 
    P1 = [*range(1,len(S)+1,3)]
    P2 = [*range(2,len(S)+1,3)]
    P_12 = P1 + P2
    
    #Obtention des triplets à partir de P1+P2 :
    R_12 = []
    for val in P_12 :
        R_12.append([list(DC3_table[1][val:val+3]), val])
    
    radix_sort(R_12,3) # On trie les triplets

    index_12 = [] # Liste des indexes de R12 trié
    order_count = 1 # Compteur pour remplir l'ordre
    recur = False # Etat de la récursion, tourné True si on a des égalités d'ordre
    for j in range(len(R_12)) : # On parcours tous les triplets triés
        index_12.append(R_12[j][1]) # ... pour lui attribuer son index depuis P_12
        DC3_table[2][R_12[j][1]] = order_count # Et on ajoute l'ordre dans la table
        if j < len(R_12)-1 : # On compare à celui d'après 
            if R_12[j][0] != R_12[j+1][0] : # On teste l'égalité des triplets pour mettre l'ordre
                order_count += 1
            else :
                recur = True # On a égalité, donc on doit relancer l'algorithme à la fin du for
        else :
            order_count += 1 # Le dernier est forcément d'ordre différent sinon on le voit sur celui d'avant

    if recur == True :
        new_S = [] # On crée T' la séquence des orders suivant l'ordre de P12
        for l in P_12 :
            new_S.append(DC3_table[2][l]) # On prend les ordres des indices de P1+2
        index_012 = DC3(new_S, P_12) # On doit donner ces deux paramètres sinon lorsqu'on revient dans le programme on a pas les bons indexes
        index_12 = []
        for ind,val in index_012 :
            DC3_table[2][ind] = val # On met à jour les ordres ...
            index_12.append(ind) # ... et on récupère les indices
    
    R_0 = [] # On crée la dernière partie à trier
    for val in P0 :
        R_0.append([[int(DC3_table[1][val]), DC3_table[2][val + 1]], val]) # On crée R0 avec son indice
    
    
    radix_sort(R_0,2) # On trie à nouveau mais des doublets cette fois
    
    index_0 = [] # Liste des indexes de R0 trié
    for k in range(len(R_0)) : # On parcours tous les doublets triés
       index_0.append(R_0[k][1]) # On récupère l'indice
    
    index_012 = [] # On crée l'index final en ordonant 0 et 1,2
    i_0 = 0 # On initialise les compteurs de chaque listes (index_0 et index_12)
    i_12 = 0
    
    index_12_dict = {} # On crée un dictionnaire des index et de leurs indice dans la liste pour accélèrer le processus de tri
    for i in range(len(index_12)) :
        index_12_dict[index_12[i]] = i
    # On commence le tri des listes
    while (i_0 < len(index_0) and i_12 < len(index_12)) : # On prends tout les éléments : on vide index 0 ou 12
        val_i0 = index_0[i_0]
        val_i12 = index_12[i_12]
        if DC3_table[1][val_i0] > DC3_table[1][val_i12] : # Cas où index 12 arrive avant index 0
            index_012.append(index_12[i_12])
            i_12 += 1
        elif DC3_table[1][val_i0] < DC3_table[1][val_i12] : # Cas où index 0 arrive avant index 12
            index_012.append(index_0[i_0])
            i_0 += 1
        else : # Cas d'égalité sur l'indice : si les 2 indexes renvoient le même nombre 
            if index_12[i_12] % 3 == 1 :
                if index_12_dict[val_i0 + 1] > index_12_dict[val_i12 + 1] : # Cas où index 12 au deuxième terme arrive avant index 0 au deuxième terme
                    index_012.append(index_12[i_12])
                    i_12 += 1
                else : # Cas où index 0 au deuxième terme arrive avant index 12 au deuxième terme
                    index_012.append(index_0[i_0])
                    i_0 += 1
            else :
                if DC3_table[1][val_i0 + 1] > DC3_table[1][val_i12 + 1] : # On teste dabord cas où index 12 + 1 arrive avant index 0 + 1
                    index_012.append(index_12[i_12])
                    i_12 += 1
                elif DC3_table[1][val_i0 + 1] < DC3_table[1][val_i12 + 1] : # Cas où index 0 + 1 arrive avant index 12 + 1
                    index_012.append(index_0[i_0])
                    i_0 += 1
                elif index_12_dict[val_i0 + 2] > index_12_dict[val_i12 + 2] : # Cas où index 12 au deuxième terme arrive avant index 0 au troisième terme
                    index_012.append(index_12[i_12])
                    i_12 += 1
                else : # Cas où index 0 au troisième terme arrive avant index 12 au deuxième terme
                    index_012.append(index_0[i_0])
                    i_0 += 1
    
    index_012.extend(index_12[i_12:]) # Si 1 des deux index est encore plein, on ajoute son contenu
    index_012.extend(index_0[i_0:])

    if int(DC3_table[1][index_012[0]]) == 0 : # On enlève le terme sentinel s'il est présent
        index_012 = index_012[1:]

    if len(P_12_base) > 0 : # Mapping sur recursion -1 si existe
        new_index_012 = []
        for n in range(len(index_012)) :
            new_index_012.append([P_12_base[index_012[n]], n]) # On transfère la valeur de base et son index pour la récursion -1
        index_012 = new_index_012
   

    return index_012 # Retourne le suffix array si dernière récursion, sinon la liste [val , index] à la récursion -1

### Fonctions de mapping :

Complexité des algorithmes :<br>
BWT : linéaire avec la taille du génome <br>
pattern_matching_bwt : Dans le pire des cas, taille du génome * taille du pattern (mais ce cas est très rare)<br>
string_location : Dans le pire des cas, taille du génome / taille du pattern (si le génome ne contient que le pattern répété)<br>
k_positioning : O(1) (la fonction sers juste à appeler les autres), donc la complexité dépend uniquement de la fonction avec la fonction la plus grande ce qui donne taille du génome * taille du pattern dans le pire des cas

In [4]:
def BWT(text,suffix_table):
    """
    Compute the BWT from the suffix table

    Args:
        T (str): string
        end_of_string (char): end of string character to append

    Return:
        bwt (str): BWT
    """
    bwt = ""
    sf_tab = suffix_table
    for i in range(len(sf_tab)):
        crt = sf_tab[i]
        bwt += text[crt-1]
    return(bwt)

def pattern_matching_BWT(S,pattern,bwt,index,somme):
    """
    Search a pattern in a String using the BWT

    Args:
        S (str): string
        pattern (str): pattern
        bwt (str) : the bwt of the text (to not compute it each time)
        index (list): the index for the chars in the bwt
        somme (list): the sum of each char in the bwt

    Return:
        bool: true if the pattern is in the string
        int : position of the first occurence of the pattern in the ordered text
        int : position of the last occurence of the pattern in the ordered text
    """
    pattern_in_S = False
    L = list(bwt)
    lpattern = list(pattern)
    start_string = -1
    end_string = -1
    ##init des valeurs utiles pour la substring search
    e = 0
    f = len(L)
    i = len(lpattern)-1
    X = lpattern[i]##correspond au dernier char du kmer, le premier a être cherché dans la string search
    for tpl in somme :
        if tpl[0]<X:
            e = tpl[1]+1 ##donne place du premier char dans la liste ordonnée
        if tpl[0]==X:
            f = tpl[1]-1 ##donne place du dernier char

    while e < f and i > 0 :
        X = lpattern[i]
        Y = lpattern[i-1]
        suite_impos = True
        r = e
        s = f
        for u in range(r,s+1):##On cherche le char suivant du pattern dans la bwt
            if(suite_impos==False):##we use the boolean to exit the loop early
                break
            if(L[u]==Y):
                suite_impos= False
                prev = 0
                idx = index[u]
                for tpl in somme :
                    if tpl[0]<Y:
                        prev = tpl[1]
                e = prev + idx ##on calcule la position du char trouvé dans la liste des chars sorted.
                start_string = e

        char_found = False
        for u in reversed(range(r,s+1)):
            if(char_found or suite_impos):
                break
            if(L[u]==Y):
                char_found = True
                prev = 0
                idx = index[u]
                for tpl in somme :
                    if tpl[0]<Y:
                        prev = tpl[1]
                f = prev + idx
                end_string = f
        if suite_impos :## this will stop the loop if no char has been found in the previous one
            break
        i -= 1
    if suite_impos:
        i = 0
        pattern_in_S = False
        return pattern_in_S,start_string,end_string
    ##dans le cas où e = f, il faut vérifier que le reste du substring est bon
    if i > 0 :
        while i > 0 :##on fait une bwt inverse "classique" en vérifiant a chaque fois que le char suivant 
                     ##est bien le même que celui du pattern
            if L[e] != lpattern[i-1]:
                break
            prev = 0
            idx = index[e]
            start_string = e
            end_string = e
            for tpl in somme :
                if tpl[0]<L[e]:
                    prev = tpl[1]

            e = prev + idx
            i -= 1
        idx = index[e]
        start_string = e
        end_string = e
    if i < 1 :
        pattern_in_S = True
    return pattern_in_S,start_string,end_string

def string_location(text,string,matches,suffix_table):
    '''
    Gives the position of each occurence of the substring in the text

    Args :
        text (string) : the text to search in
        string (string) : the substring that was search (the pattern)
        matches (tuple) : the output of the pattern matching function, containing a boolean 
            to say if an occurence was found (matches[0]), the position of the first occurence 
            in the suffix table (matches[1]) and the position of the last occurence (matches[2])
        suffix_table (list) : the ordered list of all suffixes

    Return :
        (list) : the list of all positions of the occurences of the pattern in the original text
    '''
    sft = suffix_table
    list_occur = []
    if matches[0] == False :
        #print("No occurence of the substring was found")
        list_occur.append(-1)
    else :
        for i in range(matches[1],(matches[2]+1)):##les occurences du patterns se suivent toutes dans la table des suffixes
            idx = sft[i]
            list_occur.append(idx)
            ##print(text[idx:idx+len(string)])
    return(list_occur)

def k_positioning(text,patt,bwt,suffix_table,index,somme):##permet d'obtenir la liste des positions
    '''
    Args :
        text (string) : the text to search in
        string (string) : the substring that will be searched (the pattern)
        bwt (str) : the bwt of the text (to not compute it each time)
        suffix_table (list) : the ordered list of all suffixes
        index (list): the index for the chars in the bwt
        somme (list): the sum of each char in the bwt
    
    Return :
        (list) : the list of all positions of the occurences of the pattern in the original text
    '''
    ##recuperation des positions des premiers et derniers patterns trouvés
    mat = pattern_matching_BWT(text,patt,bwt,index,somme)
    ##recupération et renvoi des positions de tout les patterns
    return string_location(text,patt,mat,suffix_table)

### Fonctions d'assemblages des kmers 

Compexité des algorithms : La fonction link_reads a une complexité maximale de la taille du génome, il est cependant quasiment impossible d'avoir une telle complexité puisque cela voudrait dire que le read est répété sur tout le génome.

In [5]:
def prepare_reads(data,k, num):
    """
    give_position of each read
    Args:
        data :List containing the treatment of kmer by mapping :
        (seq_name, kmer_pos, pos_genom_list)
        seq_name : read containing the kmer,
        kmer_pos : position of the kmer in the read,
        pos_genom_list : possible positions of the kmer in the genom
        k : length of kmers
        num : number of kmer

    Return:
        [data[0][0], pres_list] : List that contain sequence name, first, and his linking result, in second
    """
    # Creating a list of positions that regroup for the sequence the right psotions by chromosome
    read_pos=[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]] 
    pres_list = [] 
    # We go through all chromosome ... (maybe need an investigation here too)
    # ... for each chromosome because of putting them together in the same place doesn't seams to work
     
    for j in range(len(data[0][2])) :
        for kmer in range(num) :
            read_pos[j].append(data[kmer][2][j])
    
    for chro in range(len(data[0][2])) :
        pres_list.append(list(link_reads(read_pos[chro],k))) # We stock the linking of the chromosome
    
    return [data[0][0], pres_list]
    
    
def link_reads (l_pos,k):
    """
    Links the kmer of the reads
    Args:
        l_pos : list of tuple : (kmer_pos_read, kmer_pos_gen)
        k : length of kmers
    Return:
        valid : boolean of if sequence has been mapped
        val_pos(_m) : position possible of occurence of the read
        comment : type of the linking, if ""(empty) perfect linking

    """
    valid=False # true if there is a valid position
    val_pos=[] # list of valid starting position of the read in the genom
    val_pos_m=[] # list of valid starting position of the mutated reads in the genom or of reads of one kmer
    comment=""
    full=False #is true if there is the whole read at a position

    for i in l_pos[0][1] :# for each first position,we look at the last kmer to see if it can correpond with the genom but with mutation between
        if(len(l_pos)!=1):   # if there is more than 1 element in the list
            pos_gen=i+k*(len(l_pos)-1) #  position of the last kmer of the read in the genom
            if (pos_gen in l_pos[-1][1]) : # we try to find the last position in the last element of the list
                    comment="possible mutation"
                    val_pos_m.append(i)# to return the position of the read(s) in the genom
                    valid=True
        elif(len(l_pos)==1):
            if (i!=-1):#if there is only a position -1 it returns False and no position
                valid=True
                val_pos_m.append(i)
    if(valid==True):
        for i in val_pos_m :# for each first position find in previous,we add the next part to see if it exists
            cur_pos=1
            if(cur_pos!=len(l_pos)):  # if there is more than 1 element in the list
                pos_gen=i+k # next position of the kmer of the read in the genom
                while (pos_gen in l_pos[cur_pos][1]) : # we try to find the next kmer in the next list of position of kmer
                    cur_pos+=1
                    if(cur_pos==len(l_pos)): # when there is the whole read in the genom
                        val_pos.append(i)# to return the position of the read(s) in the genom
                        comment=""
                        full=True
                        break
                    else :
                        pos_gen+=k


    if(full):
        return (valid, val_pos,comment)
    else :
        return (valid, val_pos_m,comment)

### Fonction du fichier de sortie du programme

 

In [6]:
def export_result(result) :
    """
    Ecrit les résultats obtenus du mapping pour chaque read dans un fichier texte
    """
    longueur_read = 100 # Ici on sait que c'est 100, changer en detection automatique si j'ai le temps
    with open("result.txt", 'a') as f: # On ouvre le fichier résultat
            line = result[0]
            find = False
            pos = ""
            for j in range(len(result[1])) : # Si plusieurs séquences, mais obsolète depuis la version better
                if result[1][j][0] == True :
                    find = True
                    for el in result[1][j][1] :
                        """
                        Ok, là on peut se dire c'est bizarre mais pour comprendre :
                        Les chromosomes sont regroupés ensemble dans la liste donc j % 2 == 1 permet de séparer le brin sens et anti-sens
                        Et on doit retrouver le numéro du chromose dans cette liste bizarre don on divise par 2 car on a 2 brins par chromosome
                        Dans le brin anit-sens, le mapping doit se faire par rpport au brin sens pour garder une cohérence avec la méthode à comparer
                        donc on retranche au total la position et la longeur du read pour avoir sa position dans le brin sens (mais qui sera inversé ducoup !)
                        """
                        if j % 2 == 1 :
                            if result[1][j][2] == "possible mutation" :
                                pos += str(int(j/2) + 1) + " - : " + "~" + str(len(list_genom[int(j/2)][0]) - longueur_read - el + 1 ) + "\t"
                            else :
                                pos += str(int(j/2) + 1) + " - : " + str(len(list_genom[int(j/2)][0]) - longueur_read - el + 1 ) + "\t"
                        else :
                            if result[1][j][2] == "possible mutation" :
                                pos += str(int(j/2) + 1) + " + : " + "~" + str(el + 1) + "\t"
                            else :
                                pos += str(int(j/2) + 1) + " + : " + str(el + 1) + "\t"
            if find == False : # On rempli la ligne avec l'info que la séquence n'est pas mappé
                line += "\tFalse"
            else :
                line += "\tTrue" + "\t" + pos
            f.write(line + "\n")

## Appels des fonctions et déroulé du programme

In [7]:
# On donne la possibilité de continuer un mapping déja existant (au cas où le mapping serait trop long par exemple):
Continue = False

if Continue == False :
    # On initialise le fichier de sauvegarde des résultats
    with open("result.txt", 'w') as f: # On ouvre le fichier résultat
        f.write("Name_seq" + "\t" + "\t" + "Find ?" + "\t" + "Where : n° chromosome brin : start" + "\n") # On écrit le header
else : # On reprend depuis la position arrêté, changement manuel actuellement mais possibilité d'automatiser en récupérant le nombre de ligne dans le fichier résultat
    start = 200000 # Attention mettre la séquence de départ et non le n° du kmer (différent de la version de base !)

# On récupère la séquence du génome, chaque chromose étant compartimenté dans la liste
list_genom = genome_import() 

# On crée le brin inverse complémentaire du génome :
list_genom_inv = []
for data in list_genom :
    genom = Seq(data[0])
    genom = genom.reverse_complement()
    list_genom_inv.append([str(genom), data[1] + " : compl inv"])

# On récupère les reads obtenu lors du séquençage et on les découpes, chaque élément de la liste correspond à 1 kmer et num le nombre de kmer par séquence
list_reads, num = reads_import_cuts(k)

# On calcule et incorpore les DC3 aux listes

if by_DC3 == True : # Par calcul
    # Brin sens
    for i in tqdm(range(len(list_genom)), desc = "Initialisation - Calcul DC3 sens") :
        data = DC3(list_genom[i][0] + "$")
        list_genom[i].append(data)
    # Brin anti-sens
    for i in tqdm(range(len(list_genom_inv)), desc = "Initialisation - Calcul DC3 anti-sens") :
        data = DC3(list_genom_inv[i][0] + "$")
        list_genom_inv[i].append(data)
    
else : # Par import
    # Brin sens
    for i in tqdm(range(len(list_genom)) ,desc = "Initialisation - Import DC3 sens") :
        filename1 = "DC3_chrom"
        filename3 = ".txt"
        filename = filename1 + str(i+1) + filename3

        list_genom[i].append(import_DC3(filename))
    # Brin anti-sens
    for i in tqdm(range(len(list_genom)) ,desc = "Initialisation - Import DC3 anti-sens") :
        filename1 = "DC3_chrom"
        filename3 = ".txt"
        filename = filename1 + str(i+1) + "_inv" + filename3

        list_genom_inv[i].append(import_DC3(filename))

# On calcule la bwt, les index et les sommes pour chaque chromosome et on stocke en liste :  
list_bwt = []
list_index = []
list_sum = []
for i in tqdm(range(len(list_genom)), desc = "Initialisation - Listes annexes chromosome brin sens") : # Brin sens
    bwt = BWT(list_genom[i][0] + "$",list_genom[i][2])
    list_bwt.append(bwt)
    ##initialisation de l'alphabet, de l'index et du compteur total de char
    L = list(bwt)
    alphabet = ['$','A','C','G','T']
    index = []
    char_index = {}
    for char in L:
        if char not in char_index: 
            char_index[char] = 0
        index.append(char_index[char])
        char_index[char] += 1
    total = Counter(list_genom[i][0] + "$")
    som = 0
    somme = []
    for char in alphabet:
        som += total[char]
        somme.append((char,som))
    list_sum.append(somme)
    list_index.append(index)

for i in tqdm(range(len(list_genom_inv)), desc = "Initialisation - Listes annexes chromosome brin antisens") : # Brin anti-sens
    bwt = BWT(list_genom_inv[i][0] + "$",list_genom_inv[i][2])
    list_bwt.append(bwt)
    ##initialisation de l'alphabet, de l'index et du compteur total de char
    L = list(bwt)
    alphabet = ['$','A','C','G','T']
    index = []
    char_index = {}
    for char in L:
        if char not in char_index: 
            char_index[char] = 0
        index.append(char_index[char])
        char_index[char] += 1
    total = Counter(list_genom_inv[i][0] + "$")
    som = 0
    somme = []
    for char in alphabet:
        som += total[char]
        somme.append((char,som))
    list_sum.append(somme)
    list_index.append(index)

# On rentre dans l'analyse des données maintenant que les préparations du programme sont terminées :
for reads in tqdm(range(start, int(len(list_reads)/num)), desc = "Main - Read n°") :
    mapp_list = [] # On crée/vide la liste pour chaque séquence
    # On mappe :
    for genom in range(len(list_genom)) : # On merge les deux cas (sens et anti-sens) !
        for kmer in range(num) :
            pos_in_l = reads * num + kmer # On calcule la position à laquelle on se trouve dans la liste des reads
            mapp_list.append([list_reads[pos_in_l][2], k_positioning(list_genom[genom][0] + "$", list_reads[pos_in_l][0], 
                                                                     list_bwt[genom], list_genom[genom][2], list_index[genom],
                                                                     list_sum[genom])])
            mapp_list.append([list_reads[pos_in_l][2], k_positioning(list_genom_inv[genom][0] + "$", list_reads[pos_in_l][0], 
                                                                     list_bwt[genom + 15], list_genom_inv[genom][2], 
                                                                     list_index[genom + 15], list_sum[genom + 15])])
    # On crée la liste pour la liaison des kmers :
    list_liaison = []
    
    for kmer in range(num) :
        list_lia_temp = []
        for all_gen in range(len(list_genom)) :
            list_lia_temp.append(mapp_list[2*num*all_gen + kmer*2])
            list_lia_temp.append(mapp_list[2*num*all_gen + kmer*2 + 1])
        list_liaison.append([list_reads[reads * num + kmer][1], list_reads[reads * num + kmer][2], list_lia_temp])
        
    # On lie les kmers :
    result = prepare_reads(list_liaison, k, num)
    
    # On écrit les résultats dans le fichier :
    export_result(result)
    


        

Initialisation - Import sequence: 1500000it [00:55, 27028.19it/s]
Initialisation - Import DC3 anti-sens: 100%|███████████████████████████████████████████| 15/15 [00:16<00:00,  1.07s/it]
Initialisation - Listes annexes chromosome brin sens: 100%|████████████████████████████| 15/15 [00:32<00:00,  2.17s/it]
Initialisation - Listes annexes chromosome brin antisens: 100%|████████████████████████| 15/15 [00:38<00:00,  2.55s/it]
Main - Read n°: 100%|███████████████████████████████████████████████████████████| 5000/5000 [13:24:28<00:00,  9.65s/it]
